In [1]:
from charactertraining.constants import CONSTITUTION_PATH, DATA_PATH

import json
import pandas as pd

In [2]:
with open(f"{CONSTITUTION_PATH}/main.txt", 'r') as f:
    data = json.load(f)
data = pd.DataFrame(data)

In [11]:
# small - we only use questions written per trait
prompts = pd.DataFrame(columns=["category", "trait", "clarification", "messages"])
for _, row in data.iterrows():
    for question in row["questions"]:
        messages = [{"role": "user", "content": question}]
        newrow = [row["category"], row["trait"], row["clarification"], messages]
        prompts.loc[len(prompts)] = newrow
prompts.to_json(f"{DATA_PATH}/questions_main_mini.jsonl", orient="records", lines=True)

In [3]:
# we use every trait in a given category against every question
prompts = pd.DataFrame(columns=["category", "trait", "clarification", "question", "messages"])
categories = data["category"].unique()
for category in categories:
    questions = [question for sublist in data[data["category"] == category]["questions"].to_list() for question in sublist]
    ids = data[data["category"] == category]["id"].unique()
    for id in ids:
        trait = data[data["id"] == id]["trait"].item()
        clarification = data[data["id"] == id]["clarification"].item()
        for question in questions:
            messages = [{"role": "user", "content": question}]
            row = {"category": category, "trait": trait, "clarification": clarification, "question": question, "messages": messages}
            prompts.loc[len(prompts)] = row
prompts.to_json(f"{DATA_PATH}/questions_main.jsonl", orient="records", lines=True)